In [1]:
import sys
import os
import json
import boto3

In [2]:
src_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir,'src'))
src_dir

'c:\\Users\\negra\\mymlprojects\\loka\\src'

In [3]:
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [4]:
from context_retriever import retrieve_context
from context_retriever import KNOWLEDGE_BASE_ID
from orchestrator import orchestrate

In [5]:
BEDROCK_MODEL_ID = 'amazon.titan-text-lite-v1'
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

In [6]:
query = "What surgery did patient Jane Smith have?"
orchestrate(query)

HERE
RETRIEVE
You are an assistant. Use the information in the context to answer the question. If unsure, say you don't know.
Question: What surgery did patient Jane Smith have?
Context:
 Date Created: 2024-07-29 14:30:00 UTC --- Patient Information Patient ID: P-9876543 Name: Jane Smith Date of Birth: 1975-03-15 Gender: Female Address: 456 Oak Avenue, Anytown, CA 90210 Contact Number: 555-123-4567 --- Clinician Information Clinician ID: C-112233 Name: Dr. Emily White Specialization: Internal Medicine Institution: City General Hospital Contact Email: emily.white@citygen.org --- Chief Complaint Acute appendicitis, post-surgical recovery. --- History of Present Illness Ms. Smith was admitted on 2024-07-25 with severe right lower quadrant abdominal pain, nausea, and fever. Diagnostic imaging confirmed acute appendicitis. She underwent an uncomplicated laparoscopic appendectomy on the same day. Post-operatively, she had good pain control and tolerated oral intake. --- Past Medical History 

'The surgery that patient Jane Smith had was laparoscopic appendectomy.'